In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer

cancer_dataset = load_breast_cancer()

cancer_dataset_df = pd.DataFrame(cancer_dataset.data, columns=cancer_dataset.feature_names)
cancer_dataset_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from sklearn.model_selection import train_test_split
# dataset split
X_train, X_test, y_train, y_test = train_test_split(cancer_dataset.data, cancer_dataset.target, test_size=0.2, random_state=62)

print(X_train.shape , X_test.shape)

(455, 30) (114, 30)


In [ ]:
# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_wrapper.fit(X_train , y_train)

y_preds = xgb_wrapper.predict(X_test)
accuracy_score(y_test, y_preds)

0.9473684210526315

In [ ]:
# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

evals = [(X_test, y_test)]

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_wrapper.fit(X_train , y_train,  early_stopping_rounds=100,eval_set=evals, eval_metric="logloss",  verbose=True)

y_preds = xgb_wrapper.predict(X_test)
# y_pred_proba = xgb_wrapper.predict_proba(X_test)

[0]	validation_0-logloss:0.619649
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.557655
[2]	validation_0-logloss:0.5065
[3]	validation_0-logloss:0.464666
[4]	validation_0-logloss:0.426128
[5]	validation_0-logloss:0.39497
[6]	validation_0-logloss:0.366475
[7]	validation_0-logloss:0.340439
[8]	validation_0-logloss:0.318717
[9]	validation_0-logloss:0.298431
[10]	validation_0-logloss:0.282137
[11]	validation_0-logloss:0.264985
[12]	validation_0-logloss:0.252261
[13]	validation_0-logloss:0.238293
[14]	validation_0-logloss:0.228732
[15]	validation_0-logloss:0.220083
[16]	validation_0-logloss:0.210824
[17]	validation_0-logloss:0.202517
[18]	validation_0-logloss:0.194337
[19]	validation_0-logloss:0.187467
[20]	validation_0-logloss:0.18391
[21]	validation_0-logloss:0.17882
[22]	validation_0-logloss:0.173741
[23]	validation_0-logloss:0.170639
[24]	validation_0-logloss:0.165883
[25]	validation_0-logloss:0.16492
[26]	validation_0-logloss:0.160912
[2

> n_estimators는 생성할 트리의 개수로 400정도를 지정하자. 이 값은 높으면 높을 수록 성능이 향상되지만 그만큼 시간이 오래 걸린다.

> learning_rate는 GBM이 학습을 진행할 때마다 적용하는 학습률로 오류 값을 보정해 가는 데 적용되는 계수로 0.1을 사용했으며 작을 수록 오류를 많이 찾을 수 있지만 시간이 오래 걸린다.

> Max_depth는 트리의 최대 깊이로 3을 설정했으며 보통 3~10정도를 설정한다.

> early_stopping_rounds은 n_estimators가 400임으로 400번의 부스팅을 진행하는 동안 100번의 학습오류가 감소하지 않으면 더 이상 부스팅을 진행하지 않고 종료한다.

> eval_set은 evaluation 세트, 즉 검증 세트를 지정하는 것이다.  fit( ) 수행 시 반복적으로 예측 오류값을 줄일 수 있도록 학습이 진행되는데 이때 학습은 학습 데이터로 하되, 예측 오류값 평가는 eval_set로 지정된 검증 세트로 평가하는 방식이다. 학습 데이터로만 예측 오류값을 줄이게 되면 오버 피팅이 될 가능성이 높아 별도의 검증 세트를 지정하여 수행해야 한다.

> eval_metric은 검증 함수를 지정할 수 있으며 예측값 가중치 적용을 위해 logloss를 이용했다.

> 48번까지 수치가 줄다 다시 늘어 난 것을 볼 수 있으며 100회가 지나도 수치는 줄지 않아 멈추게 된다.

> 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {
    'max_depth': [3,5,7], 
    'subsample': [0.6, 0.8, 1.0]
}

grid_cv = GridSearchCV(xgb_wrapper, param_grid=param, scoring='accuracy',  cv=2, verbose=1, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.7s finished


최적 하이퍼 파라미터: 
 {'max_depth': 3, 'subsample': 0.6}
최고 예측 정확도: 0.9649


In [ ]:
from sklearn.model_selection import GridSearchCV

param = {
    'max_depth': [3,5,7], 
    'subsample': [0.4, 0.5, 0.6]
}

grid_cv = GridSearchCV(xgb_wrapper, param_grid=param,scoring='accuracy',  cv=2, verbose=1, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    1.5s finished


최적 하이퍼 파라미터: 
 {'max_depth': 3, 'subsample': 0.5}
최고 예측 정확도: 0.9693
